In [ ]:
from IPython.display import HTML, display
import torch
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
pip -q install .

In [ ]:

pip show iesta

In [ ]:
from iesta.llms.generate import Generator
import argparse
from dotenv import load_dotenv, find_dotenv

import iesta
import iesta.llms
import iesta.llms.models
from iesta.llms.models import LlamaV2, ChatGpt

import importlib
importlib.reload(iesta)
importlib.reload(iesta)

In [ ]:
%pip -q install -r ../requirements.txt

In [ ]:
all_experiments = _fetch_all_experiments()
exceptions_dict = {}# {k:[] for k in all_experiments}

exceptions_dict["liberal_chatgpt_0shot"] = [44708, 35204, 3176, 42440, 34248, 51598, 45614, 38169, 16602, 3259, 252,]
exceptions_dict["conservative_chatgpt_0shot"] = [79903, 17012, 66698, 824, 43599, 46682, 78779, 78643, 58514, 43596, 25775, 32238, 61381]
exceptions_dict["liberal_chatgpt_1shot"] = [252, 20310, 18397, 9161, 45614]
exceptions_dict["conservative_chatgpt_1shot"] = [79903, 66698, 40673, 23110, 13417, 46682, 78778, 78643, 11374, 43596, 9181, 49307, 48861,] # 10968 "Vote for me for bacon and strippers"

total = 0
for k,v in exceptions_dict.items():
    print(f"{k}; {len(v)}")
    total = total+ len(v)

total
total*100/14000

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

df = processed_toxicity_dict['liberal_llamav2_0shot_steered_medianl0.2_processed.csv']
# sns.countplot(data=df, x="success", hue="toxic_str")
counter_arr = []
for k, experiment in processed_toxicity_dict.items():
    # exp_res = {"experiment": experiment}
    
    exp_res =experiment[["success", "toxic_str"]].groupby(["success", "toxic_str"])["toxic_str"].agg([ "count"]).add_prefix('toxicity_').reset_index()
    exp_res["experiment"] = k.replace(".csv", "")
    exp_res["ideology"] = k.split("_")[0]
    exp_res["model"] = k.split("_")[1]
    exp_res["Shot"] = k.split("_")[2]
    
    counter_arr.append(exp_res)


def _apply_format(row):
    row["success_toxic"] = f"{'Response' if row['success'] else 'No Response'} w {row['toxic_str'].replace('Not ', 'Non-')} Argument"
    return row
concat_count_df = pd.concat(counter_arr, ignore_index=True, )
concat_count_df = concat_count_df.apply(_apply_format, axis=1)
concat_count_df = concat_count_df[["success_toxic", "toxicity_count", "experiment", "model"]]
concat_count_df.head()


In [ ]:
#concat_count_df.set_index("experiment")
ax = sns.barplot(concat_count_df, x="experiment", y="toxicity_count", hue="success_toxic", dodge=True)#, col="model")
#ax.plot(1955, 3600, "*", markersize=10, color="r")


In [ ]:
pip -q install  .

In [ ]:
pip show iesta

In [ ]:
import importlib
import iesta
import iesta.evaluator
importlib.reload(iesta)

importlib.reload(iesta.evaluator)
from iesta.evaluator import generation_processor, evaluator



In [ ]:
from iesta.llms.generate import Generator


In [ ]:
experiments = generation_processor._fetch_all_experiments(root="")
experiments


In [ ]:
experiment_eval_dict = {}
for experiment in experiments:
    if experiment in ["liberal_llamav2_13b_0shot" ]:
        continue
    splits = experiment.split("_")
    ideology = splits[0]
    model_type = splits[1]
    # if model_type == "chatgpt": continue
    shot = splits[2].replace("shot", "")
    steered = f"_{splits[3]}_{splits[4]}" if len(splits) >4 else ""
    experiment_eval_dict[experiment] = evaluator.Evaluator(model_type=model_type,
                                                           ideology=ideology, 
                                                           shot_num=int(shot),
                                                           steered=steered,
                                                           root_path="")
    

In [5]:
#experiment_eval_dict = {}
for experiment in experiments:
    if len(experiment_eval_dict[experiment].merged_df) == 3500:
        print(experiment)
        
    if experiment == "conservative_llamav2_1shot":
        experiment_eval_dict[experiment] = experiment_eval_dict[experiment].score_style()
        break
    

conservative_chatgpt_0shot
conservative_llamav2_0shot
conservative_llamav2_0shot_steered_meanl0.2
conservative_llamav2_0shot_steered_meanl0.5
conservative_llamav2_0shot_steered_medianl0.2
conservative_llamav2_1shot
liberal_chatgpt_0shot
liberal_chatgpt_1shot
liberal_llamav2_0shot
liberal_llamav2_0shot_steered_meanl0.2
liberal_llamav2_0shot_steered_meanl0.5
liberal_llamav2_0shot_steered_medianl0.2
liberal_llamav2_1shot


In [ ]:
import pandas as pd
corr_prompt_dict = {"liberal": [], "conservative": []}
for key, evaluator_exp in experiment_eval_dict.items():
    ideology = evaluator_exp.ideology
    model_type = evaluator_exp.model_type
    if model_type !="llamav2":
        continue
    
    corr_df = evaluator_exp.corr_toxicity_no_response
    corr_df =corr_df[corr_df["group_type"] == ""]
    corr_df = corr_df.drop('group_value', axis=1)
    corr_df = corr_df.drop('group_type', axis=1)
    def remove_ide(row):
        row['experiment'] = row['experiment'].replace(f"{ideology}_", "")
        return row
    corr_df = corr_df.apply(remove_ide, axis=1)
    corr_df = corr_df.set_index("experiment")
    corr_df = corr_df.add_suffix(f'_{ideology}')

    corr_prompt_dict[ideology].append(corr_df)

liberal = pd.concat(corr_prompt_dict["liberal"], join='outer')
conservative = pd.concat(corr_prompt_dict["conservative"])
all_corr_df = pd.concat([conservative, liberal], axis=1)
all_corr_df

In [ ]:
import pandas as pd

def get_extended_counts(experiment_eval_dict, count_type):

    count_prompt_dict = {"liberal": [], "conservative": []}
    for key, evaluator_exp in experiment_eval_dict.items():
        ideology = evaluator_exp.ideology
        model_type = evaluator_exp.model_type
        if model_type !="llamav2":
            continue
        if count_type == "toxicity":
            count_df = evaluator_exp.count_success_per_toxic_df
        elif count_type == "prompt":
            count_df = evaluator_exp.count_success_per_prompt_df
        else:
            count_df = evaluator_exp.count_success_per_ideology_prompt_df
        count_prompt_df = count_df[["fail_perc"]].rename(columns= {"fail_perc": key.replace(f"{ideology}_", "")}).T
        count_prompt_df = count_prompt_df.drop('All', axis=1)
        count_prompt_df = count_prompt_df.add_suffix(f'_{ideology}')
        count_prompt_df.score_style()
        count_prompt_dict[ideology].append(count_prompt_df)
        break

    liberal = pd.concat(count_prompt_dict["liberal"])
    conservative = pd.concat(count_prompt_dict["conservative"])
    all_prompt_extended_df = pd.concat([conservative, liberal], axis=1)
    return all_prompt_extended_df

In [ ]:
ideology_prompt_count = get_extended_counts(experiment_eval_dict, "ideology_prompt")

In [ ]:
toxicity_prompt_count = get_extended_counts(experiment_eval_dict, "toxicity")#.to_csv("data/out/toxicity_success_extended.csv")

In [ ]:
count_extended_df = pd.concat([toxicity_prompt_count, ideology_prompt_count], axis=1)
count_extended_df[['Toxic_conservative', 'Not Toxic_conservative', 'True_conservative', 'False_conservative',
                   'Toxic_liberal', 'Not Toxic_liberal', 'True_liberal', 'False_liberal']].to_csv("data/out/failed_to_respond_perc.csv")